In [1]:
%load_ext autoreload
%autoreload 2

In [55]:
from collections import Counter
import pickle
import re

import dill
import penman
import stanza

In [36]:
with open("/projects/flow_graphs/data/risec/amr.pkl", "rb") as f:
    amr_data = pickle.load(f)

In [14]:
stanza_nlp = stanza.Pipeline(lang="en")

2022-06-16 14:14:34 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-06-16 14:14:34 INFO: Use device: gpu
2022-06-16 14:14:34 INFO: Loading: tokenize
2022-06-16 14:14:40 INFO: Loading: pos
2022-06-16 14:14:41 INFO: Loading: lemma
2022-06-16 14:14:41 INFO: Loading: depparse
2022-06-16 14:14:41 INFO: Loading: sentiment
2022-06-16 14:14:42 INFO: Loading: constituency
2022-06-16 14:14:43 INFO: Loading: ner
2022-06-16 14:14:44 INFO: Done loading processors!


In [13]:
doc_idx = 0
sent_idx = 1

graph = amr_data[doc_idx][sent_idx]["graph"]
text = amr_data[doc_idx][sent_idx]["text"]
print(text)
print(penman.encode(graph))

 2) Whisk in egg yolks and cook until light custard forms; do not boil.
(z1 / and~e.5
    :li 2
    :op1 (z2 / whisk-01
             :mode imperative
             :arg0 (z3 / you)
             :arg1 (z4 / yolk~e.4
                       :mod (z5 / egg~e.3)))
    :op2 (z6 / cook-01~e.6
             :mode imperative
             :arg0 z3
             :time~e.2 (z7 / until~e.7
                           :op1 (z8 / form~e.10
                                    :mod (z9 / custard~e.9
                                             :arg1-of (z10 / light-06~e.8)))))
    :op3 (z11 / boil-01~e.13
              :polarity~e.12 -~e.12
              :mode~e.11 imperative
              :arg0 z3))


## EDA: what is unaligned?

In [48]:
unaligned_triples = []
unaligned_instances = []

node_pattern = re.compile(r"z\d+")
pbf_pattern = re.compile(r"\w+-(\d){2}")


def should_be_unaligned(triple):
    return bool(node_pattern.fullmatch(triple[0])) and bool(node_pattern.match(triple[2]))


def is_propbank_frame(node_str):
    return bool(pbf_pattern.fullmatch(node_str))


for doc in amr_data:
    for sentence in doc:
        amr_graph = sentence["graph"]
        if amr_graph is None:
            continue
        alignments = penman.surface.alignments(amr_graph)
        for triple in amr_graph.triples:
            if triple not in alignments and not should_be_unaligned(triple):
                unaligned_triples.append(triple)
                if triple[1] == ":instance":
                    unaligned_instances.append(triple)

In [32]:
Counter([triple[1] for triple in unaligned_triples])

Counter({':instance': 1905,
         ':li': 801,
         ':mode': 235,
         ':quant': 66,
         ':polarity': 14,
         ':frequency': 4,
         ':op1': 10,
         ':op2': 3,
         ':op3': 1,
         ':op4': 1})

In [54]:
[triple for triple in unaligned_triples if triple[1] == ":mode" if triple[2] != "imperative"]

[]

In [52]:
len(unaligned_instances), len(
    [instance for instance in unaligned_instances if is_propbank_frame(instance[2])]
)

(1905, 743)